# End of game measures
This `Python` notebook takes the anonymized data and computes population-level measures for each game.

In [3]:
%pylab inline
import json
import numpy as np
import pandas as pd
import glob
import itertools
from sklearn.decomposition import PCA
from scipy import stats

from helpers import shuffle

Populating the interactive namespace from numpy and matplotlib


In [4]:
output_dir = "../results-anonymized/pilot/"
files = glob.glob(output_dir+'block_*_pilot.json')
files

['../results-anonymized/pilot/block_20200505_pilot.json',
 '../results-anonymized/pilot/block_20200507_pilot.json',
 '../results-anonymized/pilot/block_20200624_pilot.json',
 '../results-anonymized/pilot/block_20200626_pilot.json',
 '../results-anonymized/pilot/block_20200506_pilot.json']

In [5]:
blocks = []
for file in files:
    with open(file) as f:
        blocks.append(json.load(f))

In [18]:
# Enumerate clues to be used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

# Enumerate end-of-game survey questions to be used in polarization analysis
assessments = ['appearance_1', 'appearance_2', 
               'clothing_1', 'clothing_2',
               'suspect_1', 'suspect_2', 'suspect_3',
               'tool_1', 'tool_2', 
               'vehicle_1', 'vehicle_2']    
    
def compute_single_point_measures(game):
    """ 
    Compute the game-level measures 
    
    "Games" in this experiment contain both a treatment and control condition
    and these must be properly separated from one another.
    
    """
    # Form end-of-game survey responses into a dataframe
    collector = {}
    for p, k in game['players'].items():
        try:
            collector[k['data.position']] = k['data.caseMade']
        except:
            print('%s did not complete the post-game survey' %k['data.position'])
    responses = pd.DataFrame(collector).T.sort_index()

    # Form final notebook states into a dataframe
    final_adoptions = pd.DataFrame(data=0, index=responses.index, columns=t_spokes+c_spokes)
    for p, k in game['players'].items():
        for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
            final_adoptions.loc[k['data.position'], clue_id] = 1


    # Determine the number of datapoints to be used in polarization analysis
    # if there are missing responses, need to compare equal sized datasets
    t_responses = [pos for pos in responses.index if pos.startswith('t')]
    c_responses = [pos for pos in responses.index if pos.startswith('c')] 
    # use whichever condition has fewer responses to set the sample size
    n_used = min(len(t_responses), len(c_responses))


    def process_subset(subset, spokes):
        """ compute a result on the selected subset of the data """
        sub_res = {}
        
        # select the subset of the survey responses that will be used in the subset analysis
        sub_survey = responses.loc[subset, assessments]
        
        # survey PC1 
        pca = PCA(n_components=1)
        pca.fit(sub_survey)  
        sub_res['survey PC1'] = pca.explained_variance_ratio_

        # survey similarity percentiles
        survey_corrs = sub_survey.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['survey 5% similarity'], sub_res['survey 95% similarity'] = np.percentile(
            survey_corrs, [5, 95])
        
        # select the subset of the behavioral responses that will be used in the subset analysis
        sub_adopt = final_adoptions.loc[subset, spokes]
        
        # final-state PC1
        pca = PCA(n_components=1)
        pca.fit(sub_adopt)  
        sub_res['spoke PC1'] = pca.explained_variance_ratio_
        
        # final state similarity percentiles
        spoke_corrs = sub_adopt.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['spoke 5% similarity'], sub_res['spoke 95% similarity'] = np.percentile(
            spoke_corrs, [5, 95])
        
        # compute the expected values for the given level of adoption
        # by shuffling the clues between individuals 
        # (preserving the number of clues each individual holds, 
        # and the number of individuals holding each clue)
        # do this a number of times and average the result
        e95 = []
        e5 = []
        ePC1 = []
        for _ in range(100):
            shuffle_adopt = pd.DataFrame(index=sub_adopt.index,
                                         columns=sub_adopt.columns,
                                         data=shuffle(sub_adopt.values, n=500))

            n_agents = len(shuffle_adopt.index)
            corrs = shuffle_adopt.astype(float).T.corr().mask(np.tri(n_agents, n_agents, 0, dtype='bool')).stack()
            e95.append(np.percentile(corrs, 95))
            e5.append(np.percentile(corrs, 5))

            pca = PCA(n_components=1)
            pca.fit(shuffle_adopt)
            ePC1.append(pca.explained_variance_ratio_[0])
        
        # compute the net effect of (interdependent or independent) diffusion 
        # over chance distribution of the same clues
        sub_res['net spoke PC1'] = sub_res['spoke PC1'] - np.mean(ePC1)
        sub_res['net spoke 95% similarity'] = sub_res['spoke 95% similarity'] - np.mean(e95)
        sub_res['net spoke 5% similarity'] = sub_res['spoke 5% similarity'] - np.mean(e5)
        
        return sub_res
        
        
    # For each subset of size 'n_used', compute a result. 
    # In most cases there are no missing responses, so just compute on the complete set
    t_collector = []
    for subset in itertools.combinations(t_responses, r=n_used):
        t_collector.append(process_subset(subset, t_spokes))

    # The recorded result is the average over all subsets
    if len(t_collector) > 1:
        print('Averaging over %i combinations for treatment case'%len(t_collector))
    t_result = pd.DataFrame(t_collector).mean()

    # Compute average for confidence and consensus measures on all submissions
    t_result['confidence'] = responses.loc[t_responses, 'confidence'].mean()
    t_result['consensus'] = responses.loc[t_responses, 'consensus'].mean()    
    t_result['id'] = game['_id']
    
    # Perform the same analysis as above for the control condition
    c_collector = []
    for subset in itertools.combinations(c_responses, r=n_used):
        c_collector.append(process_subset(subset, c_spokes))

    if len(c_collector) > 1:
        print('Averaging over %i combinations for control case'%len(c_collector))
    c_result = pd.DataFrame(c_collector).mean()

    c_result['confidence'] = responses.loc[c_responses, 'confidence'].mean()
    c_result['consensus'] = responses.loc[c_responses, 'consensus'].mean()     

    c_result['id'] = game['_id']
    
    #pd.merge(t_result, c)result, suffixes=(' (inter)', ' (indep)'))
    result = pd.concat([t_result, c_result], keys=['inter', 'indep'])
    #result['game_id']=game['createdAt'].split('_')[0].replace('-','_').replace(':','_').replace('.','_')
    return result

def compute_block(block):
    results_collector = []
    network_collector = []
    for name, game in block.items():
        network_collector.append('caveman' if 'caveman' in name else 'dodec')
        results_collector.append(compute_single_point_measures(game))
        
    result = pd.concat(results_collector, keys=network_collector)
    return result

In [55]:
measurements = pd.concat([compute_block(block) for block in blocks], axis=1)
#measurements.to_csv(output_dir+"end_of_game_measurements.csv")
measurements

0                  1  \
caveman indep confidence                             53.4              57.55   
              consensus                             55.15              54.55   
              id                        Bd3w7KBP8ZKKMBxhj  5wifJzSKTJuDtQkzS   
              net spoke 5% similarity            -0.18959          -0.209574   
              net spoke 95% similarity           0.312202           0.314034   
              net spoke PC1                      0.206754           0.240229   
              spoke 5% similarity                -0.41833          -0.313527   
              spoke 95% similarity               0.806832           0.900423   
              spoke PC1                          0.382635           0.422482   
              survey 5% similarity              -0.725347          -0.621686   
              survey 95% similarity              0.747789           0.723486   
              survey PC1                         0.359687           0.440281   
        inter confidence                             53.5              52.35   
              consensus                              51.6               57.1   
              id                        Bd3w7KBP8ZKKMBxhj  5wifJzSKTJuDtQkzS   
              net spoke 5% similarity           -0.135521          -0.172148   
              net spoke 95% similarity           0.378661           0.293366   
              net spoke PC1                      0.320206           0.209634   
              spoke 5% similarity               -0.340905          -0.430331   
              spoke 95% similarity               0.896421           0.783046   
              spoke PC1                            0.5056           0.397125   
              survey 5% similarity              -0.553209          -0.539193   
              survey 95% similarity              0.873783           0.686564   
              survey PC1                         0.488426           0.354293   
dodec   indep confidence                            43.35               50.5   
              consensus                              42.7               50.6   
              id                        xXvsHQvoQC4LsoyCi  acKghgvvb3kewvMd9   
              net spoke 5% similarity           -0.177264          -0.143287   
              net spoke 95% similarity           0.196357           0.142416   
              net spoke PC1                     0.0872461             0.1167   
              spoke 5% similarity               -0.293518          -0.318754   
              spoke 95% similarity                0.71743            0.67082   
              spoke PC1                          0.342774           0.374724   
              survey 5% similarity              -0.492309          -0.485883   
              survey 95% similarity               0.57503           0.739458   
              survey PC1                         0.279186           0.419603   
        inter confidence                            59.35               58.3   
              consensus                              53.5              54.45   
              id                        xXvsHQvoQC4LsoyCi  acKghgvvb3kewvMd9   
              net spoke 5% similarity           -0.161001          -0.129372   
              net spoke 95% similarity            0.18125          0.0120898   
              net spoke PC1                        0.1187          0.0579818   
              spoke 5% similarity               -0.334691          -0.138534   
              spoke 95% similarity               0.716328           0.642071   
              spoke PC1                          0.301137           0.249915   
              survey 5% similarity              -0.549573          -0.351835   
              survey 95% similarity              0.746668           0.690122   
              survey PC1                         0.417522           0.311937   

                                                        2                  3  \
caveman indep confidence                              N

In [62]:
measurements_flat = measurements.unstack(level=[0,1])
measurements_flat.loc['t'] = [1 if c[2]=="inter" else 0 for c in measurements_flat.columns]
measurements_flat.loc['network'] = [1 if c[1]=="caveman" else 0 for c in measurements_flat.columns]
measurements_flat.columns = ["GT_" + str(measurements_flat.loc['id'][c]) + ("_1" if c[2]=="inter" else "_0") for c in measurements_flat.columns]
measurements_flat.T.to_csv(output_dir+"end_of_game_measurements.csv")
measurements_flat.T

,confidence,consensus,id,net spoke 5% similarity,net spoke 95% similarity,net spoke PC1,spoke 5% similarity,spoke 95% similarity,spoke PC1,survey 5% similarity,survey 95% similarity,survey PC1,t,network
GT_Bd3w7KBP8ZKKMBxhj_0,53.4,55.15,Bd3w7KBP8ZKKMBxhj,-0.18959,0.312202,0.206754,-0.41833,0.806832,0.382635,-0.725347,0.747789,0.359687,0,1
GT_Bd3w7KBP8ZKKMBxhj_1,53.5,51.6,Bd3w7KBP8ZKKMBxhj,-0.135521,0.378661,0.320206,-0.340905,0.896421,0.5056,-0.553209,0.873783,0.488426,1,1
GT_xXvsHQvoQC4LsoyCi_0,43.35,42.7,xXvsHQvoQC4LsoyCi,-0.177264,0.196357,0.0872461,-0.293518,0.71743,0.342774,-0.492309,0.57503,0.279186,0,0
GT_xXvsHQvoQC4LsoyCi_1,59.35,53.5,xXvsHQvoQC4LsoyCi,-0.161001,0.18125,0.1187,-0.334691,0.716328,0.301137,-0.549573,0.746668,0.417522,1,0
GT_5wifJzSKTJuDtQkzS_0,57.55,54.55,5wifJzSKTJuDtQkzS,-0.209574,0.314034,0.240229,-0.313527,0.900423,0.422482,-0.621686,0.723486,0.440281,0,1
GT_5wifJzSKTJuDtQkzS_1,52.35,57.1,5wifJzSKTJuDtQkzS,-0.172148,0.293366,0.209634,-0.430331,0.783046,0.397125,-0.539193,0.686564,0.354293,1,1
GT_acKghgvvb3kewvMd9_0,50.5,50.6,acKghgvvb3kewvMd9,-0.143287,0.142416,0.1167,-0.318754,0.67082,0.374724,-0.485883,0.739458,0.419603,0,0
GT_acKghgvvb3kewvMd9_1,58.3,54.45,acKghgvvb3kewvMd9,-0.129372,0.0120898,0.0579818,-0.138534,0.642071,0.249915,-0.351835,0.690122,0.311937,1,0
GT_nan_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
GT_nan_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


0                     \
                                    caveman                      
                                      indep              inter   
confidence                             53.4               53.5   
consensus                             55.15               51.6   
id                        Bd3w7KBP8ZKKMBxhj  Bd3w7KBP8ZKKMBxhj   
net spoke 5% similarity            -0.18959          -0.135521   
net spoke 95% similarity           0.312202           0.378661   
net spoke PC1                      0.206754           0.320206   
spoke 5% similarity                -0.41833          -0.340905   
spoke 95% similarity               0.806832           0.896421   
spoke PC1                          0.382635             0.5056   
survey 5% similarity              -0.725347          -0.553209   
survey 95% similarity              0.747789           0.873783   
survey PC1                         0.359687           0.488426   
t                                         0                  1   

                                                                \
                                      dodec                      
                                      indep              inter   
confidence                            43.35              59.35   
consensus                              42.7               53.5   
id                        xXvsHQvoQC4LsoyCi  xXvsHQvoQC4LsoyCi   
net spoke 5% similarity           -0.177264          -0.161001   
net spoke 95% similarity           0.196357            0.18125   
net spoke PC1                     0.0872461             0.1187   
spoke 5% similarity               -0.293518          -0.334691   
spoke 95% similarity                0.71743           0.716328   
spoke PC1                          0.342774           0.301137   
survey 5% similarity              -0.492309          -0.549573   
survey 95% similarity               0.57503           0.746668   
survey PC1                         0.279186           0.417522   
t                                         0                  1   

                                          1                     \
                                    caveman                      
                                      indep              inter   
confidence                            57.55              52.35   
consensus                             54.55               57.1   
id                        5wifJzSKTJuDtQkzS  5wifJzSKTJuDtQkzS   
net spoke 5% similarity           -0.209574          -0.172148   
net spoke 95% similarity           0.314034           0.293366   
net spoke PC1                      0.240229           0.209634   
spoke 5% similarity               -0.313527          -0.430331   
spoke 95% similarity               0.900423           0.783046   
spoke PC1                          0.422482           0.397125   
survey 5% similarity              -0.621686          -0.539193   
survey 95% similarity              0.723486           0.686564   
survey PC1                         0.440281           0.354293   
t                                         0                  1   

                                                                     2        \
                                      dodec                    caveman         
                                      indep              inter   indep inter   
confidence                             50.5               58.3     NaN   NaN   
consensus                              50.6              54.45     NaN   NaN   
id                        acKghgvvb3kewvMd9  acKghgvvb3kewvMd9     NaN   NaN   
net spoke 5% similarity           -0.143287          -0.129372     NaN   NaN   
net spoke 95% similarity           0.142416          0.0120898     NaN   NaN   
net spoke PC1                        0.1167          0.0579818     NaN   NaN   
spoke 5% similarity               -0.318754          -0.138534     NaN   NaN   
spoke 95% similarity                0.67082           0.642071     NaN 

In [10]:
# todo: drop id row for subsequent analysis
measurements.drop('id', level=1, inplace=True)

In [6]:
def bootstrap_mean(l, q=(2.5, 97.5), n=1000):
    "Basic bootstrap confidence intervals (q) with n resamples"
    return np.percentile([np.mean(np.random.choice(l, size=len(l))) for _ in range(n)], q=q)


def mean_result(measure1, _):
    return measure1.mean()

def mean_lowerbound(measure1, _):
    return bootstrap_mean(measure1)[0]

def mean_upperbound(measure1, _):
    return bootstrap_mean(measure1)[1]

def effect_size(measure1, measure2):
    return (measure1 - measure2).mean()

def effect_lowerbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[0]

def effect_upperbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[1]

def effect_p_val(measure1, measure2):
    return stats.ttest_rel(measure1, measure2)[1]


def make_table(measurements, func):
    rows = measurements.index.levels[2]
    cols = np.unique(measurements.index.droplevel(2))
    res = pd.DataFrame(index=rows, columns=cols)
    for row in rows:
        for col in cols:
            comparison = measurements.loc[col+tuple([row])].dropna()
            reference = measurements.loc[('dodec', 'indep')+tuple([row])][comparison.index]
            res.at[row, col] = func(comparison, reference)
    return res

In [7]:
res = make_table(measurements, mean_result)
res

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,52.2833,56.7667,48.99,53.38
consensus,52.7667,58.3333,49.07,53.02
net spoke 5% similarity,-0.173996,-0.122736,-0.0678498,-0.0972783
net spoke 95% similarity,0.282875,0.382337,0.117987,0.100039
net spoke PC1,0.199264,0.24732,0.0596908,0.0768571
spoke 5% similarity,-0.342128,-0.353304,-0.177564,-0.245504
spoke 95% similarity,0.83705,0.893156,0.698852,0.646924
spoke PC1,0.404108,0.43493,0.299288,0.269057
survey 5% similarity,-0.6126,-0.556754,-0.395819,-0.469243
survey 95% similarity,0.729252,0.788166,0.711058,0.683996


In [8]:
res_lowerbound = make_table(measurements, mean_lowerbound)
res_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,45.9,52.35,45.4527,48.22
consensus,48.6,51.6,44.5823,51.51
net spoke 5% similarity,-0.206095,-0.174348,-0.134877,-0.139577
net spoke 95% similarity,0.221379,0.296337,0.0704022,0.0500837
net spoke PC1,0.154448,0.210815,0.0265373,0.0535128
spoke 5% similarity,-0.41833,-0.430331,-0.274767,-0.324874
spoke 95% similarity,0.803894,0.783046,0.648923,0.617592
spoke PC1,0.382635,0.397125,0.253144,0.252542
survey 5% similarity,-0.725347,-0.57786,-0.505114,-0.543157
survey 95% similarity,0.71648,0.686564,0.639585,0.649199


In [9]:
res_upperbound = make_table(measurements, mean_upperbound)
res_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,57.55,64.45,52.64,58.21
consensus,55.15,66.3,54.08,53.9808
net spoke 5% similarity,-0.129559,-0.0542692,-0.00655293,-0.0527819
net spoke 95% similarity,0.314388,0.472725,0.165781,0.148832
net spoke PC1,0.237277,0.284503,0.0950159,0.100201
spoke 5% similarity,-0.294525,-0.288675,-0.0690318,-0.166135
spoke 95% similarity,0.900423,1,0.75821,0.683866
spoke PC1,0.422482,0.5056,0.345432,0.288255
survey 5% similarity,-0.490766,-0.539193,-0.277114,-0.394175
survey 95% similarity,0.747789,0.873783,0.760272,0.722865


In [10]:
effect = make_table(measurements, effect_size)
effect

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,2.4,6.88333,0,4.39
consensus,3.15,8.71667,0,3.95
net spoke 5% similarity,-0.0552523,-0.00399298,0,-0.0294285
net spoke 95% similarity,0.124931,0.224392,0,-0.017948
net spoke PC1,0.128887,0.176943,0,0.0171663
spoke 5% similarity,-0.14851,-0.159686,0,-0.0679407
spoke 95% similarity,0.106442,0.162549,0,-0.0519284
spoke PC1,0.0675537,0.0983758,0,-0.030231
survey 5% similarity,-0.198728,-0.142882,0,-0.0734243
survey 95% similarity,0.0466303,0.105545,0,-0.0270628


In [12]:
effect_pval = make_table(measurements.dropna(axis=1), effect_p_val)
effect_pval

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0.736415,0.114478,NaN,0.229662
consensus,0.631243,0.0193449,NaN,0.575628
net spoke 5% similarity,0.123989,0.851566,NaN,0.258454
net spoke 95% similarity,0.0290606,0.0641137,NaN,0.183979
net spoke PC1,0.00858431,0.0533355,NaN,0.805282
spoke 5% similarity,0.263095,0.191935,NaN,0.870416
spoke 95% similarity,0.251753,0.0240253,NaN,0.3424
spoke PC1,0.105343,0.138205,NaN,0.143659
survey 5% similarity,0.0242238,0.237789,NaN,0.846876
survey 95% similarity,0.536669,0.413217,NaN,0.69924


In [13]:
effect_lowerbound = make_table(measurements, effect_lowerbound)
effect_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,-9.9,1.85,0,-0.63
consensus,-6.95,6.5,0,-1.83
net spoke 5% similarity,-0.0879529,-0.0313625,0,-0.0988714
net spoke 95% similarity,0.090991,0.14901,0,-0.0857719
net spoke PC1,0.11493,0.0924536,0,-0.0235506
spoke 5% similarity,-0.325946,-0.320095,0,-0.180523
spoke 95% similarity,0.00032269,0.112226,0,-0.126495
spoke PC1,0.039861,0.022401,0,-0.075083
survey 5% similarity,-0.233037,-0.314437,0,-0.188819
survey 95% similarity,-0.0168953,-0.0528939,0,-0.101711


In [14]:
effect_upperbound = make_table(measurements, effect_upperbound)
effect_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,10.05,10.15,0,11.03
consensus,12.45,10.75,0,9.73
net spoke 5% similarity,-0.0146944,0.0320466,0,0.0137319
net spoke 95% similarity,0.165533,0.342338,0,0.0426878
net spoke PC1,0.152815,0.22852,0,0.052525
spoke 5% similarity,0.00522697,-0.0687837,0,0.0848799
spoke 95% similarity,0.229602,0.196429,0,-0.00226208
spoke PC1,0.115042,0.162826,0,0.0222326
survey 5% similarity,-0.135803,-0.0533101,0,0.0475719
survey 95% similarity,0.172759,0.298753,0,0.0733221


In [15]:
export_table = pd.DataFrame()
export_table["Result Mean"] = res.unstack()
export_table["Result Lower Bound"] = res_lowerbound.unstack()
export_table["Result Upper Bound"] = res_upperbound.unstack()
export_table["Effect Size"] = effect.unstack()
export_table["Effect Upper Bound"] = effect_upperbound.unstack()
export_table["Effect Lower Bound"] = effect_lowerbound.unstack()
export_table["Effect P Value"] = effect_pval.unstack()


export_table.to_csv(output_dir+"end_of_game_effects_summary.csv")
export_table

Result Mean Result Lower Bound  \
(caveman, indep) confidence                   52.2833               45.9   
                 consensus                    52.7667               48.6   
                 net spoke 5% similarity    -0.173996          -0.206095   
                 net spoke 95% similarity    0.282875           0.221379   
                 net spoke PC1               0.199264           0.154448   
                 spoke 5% similarity        -0.342128           -0.41833   
                 spoke 95% similarity         0.83705           0.803894   
                 spoke PC1                   0.404108           0.382635   
                 survey 5% similarity         -0.6126          -0.725347   
                 survey 95% similarity       0.729252            0.71648   
                 survey PC1                  0.368261           0.304816   
(caveman, inter) confidence                   56.7667              52.35   
                 consensus                    58.3333               51.6   
                 net spoke 5% similarity    -0.122736          -0.174348   
                 net spoke 95% similarity    0.382337           0.296337   
                 net spoke PC1                0.24732           0.210815   
                 spoke 5% similarity        -0.353304          -0.430331   
                 spoke 95% similarity        0.893156           0.783046   
                 spoke PC1                    0.43493           0.397125   
                 survey 5% similarity       -0.556754           -0.57786   
                 survey 95% similarity       0.788166           0.686564   
                 survey PC1                  0.423766           0.354293   
(dodec, indep)   confidence                     48.99            45.4527   
                 consensus                      49.07            44.5823   
                 net spoke 5% similarity   -0.0678498          -0.134877   
                 net spoke 95% similarity    0.117987          0.0704022   
                 net spoke PC1              0.0596908          0.0265373   
                 spoke 5% similarity        -0.177564          -0.274767   
                 spoke 95% similarity        0.698852           0.648923   
                 spoke PC1                   0.299288           0.253144   
                 survey 5% similarity       -0.395819          -0.505114   
                 survey 95% similarity       0.711058           0.639585   
                 survey PC1                  0.318089           0.266843   
(dodec, inter)   confidence                     53.38              48.22   
                 consensus                      53.02              51.51   
                 net spoke 5% similarity   -0.0972783          -0.139577   
                 net spoke 95% similarity    0.100039          0.0500837   
                 net spoke PC1              0.0768571          0.0535128   
                 spoke 5% similarity        -0.245504          -0.324874   
                 spoke 95% similarity        0.646924           0.617592   
                 spoke PC1                   0.269057           0.252542   
                 survey 5% similarity       -0.469243          -0.543157   
                 survey 95% similarity       0.683996           0.649199   
                 survey PC1                  0.342388           0.301875   

                                          Result Upper Bound Effect Size  \
(caveman, indep) confidence                            57.55         2.4   
                 consensus                             55.15        3.15   
                 net spoke 5% similarity           -0.129559  -0.0552523   
                 net spoke 95% similarity           0.314388    0.124931   
                 net spoke PC1                      0.237277    0.128887   
                 spoke 5% similarity               -0.294525    -0.14851   
                 spoke 95% similarity               0.900423    0.106442   
             